<a href="https://colab.research.google.com/github/zhaocaiQ/Git-Tutorial/blob/master/keyword_query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#데이터추출 쿼리

##pymysql설치

In [ ]:
pip install pymysql

In [ ]:
import pymysql
#데이터베이스 연동
conn = pymysql.connect(host='tkt-db-ko.cnirlhwrm55r.ap-northeast-2.rds.amazonaws.com', port=3306, user='admin', password='tkt202202', db='topic_keywords_db', charset='utf8')
cursor = conn.cursor()

In [ ]:
from datetime import datetime
date = datetime.today().strftime("%Y-%m-%d")

In [ ]:
print(date)

2022-02-27


#daily_top25

In [ ]:
cursor.execute("SELECT keyword, weight FROM keyword WHERE c_date = curdate() ORDER BY weight DESC LIMIT 25;")
result = cursor.fetchall()
#오늘에 대한 정보 없을 시 어제날짜로 출력
if len(result) == 0:
  cursor.execute("SELECT keyword, weight FROM keyword WHERE c_date = date_add(curdate(), interval -1 day) ORDER BY weight DESC LIMIT 25;")
  result = cursor.fetchall()
print(result)

(('우크라', 35), ('러시아', 28), ('단일', 18), ('결렬', 14), ('윤석열', 14), ('협상', 14), ('키예프', 10), ('금융', 10), ('온화', 9), ('전쟁', 9), ('안철수', 9), ('대선', 8), ('일교차', 8), ('제재', 8), ('산불', 8), ('확진', 8), ('책임', 7), ('강원', 7), ('사태', 7), ('유럽', 7), ('국제', 7), ('지원', 7), ('휴일', 6), ('반전', 6), ('이재명', 6))


##weekly_top25

In [ ]:
cursor.execute("SELECT keyword, CAST(SUM(weight) AS SIGNED) FROM keyword WHERE DATE(c_date) >= ADDDATE(curdate(), - WEEKDAY(curdate())) AND DATE(c_date) <= ADDDATE(curdate(), - WEEKDAY(curdate())+ 6) GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 25;")
result = cursor.fetchall()
#이번주에 대한 정보가 없을시 지난주에 대한 정보 출력
if len(result) == 0:
  cursor.execute("SELECT keyword, CAST(SUM(weight) AS SIGNED) FROM keyword WHERE DATE(c_date) >= ADDDATE(date_add(curdate(), interval -1 day), - WEEKDAY(date_add(curdate(), interval -1 day))) AND DATE(c_date) <= ADDDATE(date_add(curdate(), interval -1 day), - WEEKDAY(date_add(curdate(), interval -1 day))+ 6) GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 25;")
  result = cursor.fetchall()
print(result)

(('우크라', 63), ('러시아', 46), ('키예프', 34), ('윤석열', 23), ('단일', 22), ('강풍', 19), ('산불', 19), ('제재', 17), ('확진', 17), ('대선', 16), ('항전', 14), ('전쟁', 14), ('결렬', 14), ('협상', 14), ('러시아군', 12), ('최다', 12), ('이재명', 11), ('푸틴', 11), ('금융', 10), ('방역', 10), ('정치', 10), ('안철수', 9), ('사망자', 9), ('수도', 9), ('역대', 9))


##monthly_top25

In [ ]:
cursor.execute("SELECT keyword, CAST(SUM(weight) AS SIGNED) FROM keyword WHERE date_format(c_date, '%Y-%m') = date_format(curdate(), '%Y-%m') GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 25;")
result = cursor.fetchall()
#이번달에 대한 정보가 없을시 지난달에 대한 정보 출력
if len(result) == 0:
  cursor.execute("SELECT keyword, CAST(SUM(weight) AS SIGNED) FROM keyword WHERE date_format(c_date, '%Y-%m') = date_format((curdate() - INTERVAL 1 MONTH), '%Y-%m') GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 25;")
  result = cursor.fetchall()
print(result)

(('우크라', 63), ('러시아', 46), ('키예프', 34), ('윤석열', 23), ('단일', 22), ('강풍', 19), ('산불', 19), ('제재', 17), ('확진', 17), ('대선', 16), ('항전', 14), ('전쟁', 14), ('결렬', 14), ('협상', 14), ('러시아군', 12), ('최다', 12), ('이재명', 11), ('푸틴', 11), ('금융', 10), ('방역', 10), ('정치', 10), ('안철수', 9), ('사망자', 9), ('수도', 9), ('역대', 9))


##daily_chart

In [ ]:
#오늘 top1~25출력
cursor.execute("SELECT keyword, weight FROM keyword WHERE c_date = curdate() ORDER BY weight DESC LIMIT 25;")
today = dict(cursor.fetchall())
if len(today) != 0:
  #전날 top1~25출력
  cursor.execute("SELECT keyword, weight FROM keyword WHERE c_date = date_add(curdate(), interval -1 day) ORDER BY weight DESC LIMIT 25;")
  yesterday = dict(cursor.fetchall())
else:
  #오늘 정보가 없을 시 어제,그저께 정보로 데이터 출력
  #어제 top1~25출력
  cursor.execute("SELECT keyword, weight FROM keyword WHERE c_date = date_add(curdate(), interval -1 day) ORDER BY weight DESC LIMIT 25;")
  today = dict(cursor.fetchall()) 
  #그저께 top1~25출력
  cursor.execute("SELECT keyword, weight FROM keyword WHERE c_date = date_add(curdate(), interval -2 day) ORDER BY weight DESC LIMIT 25;")
  yesterday = dict(cursor.fetchall())

daily_chart = []
for i in today.keys():
    if i in yesterday.keys():
      if today[i] > yesterday[i]:
          daily_chart.append([i, today[i], "up"])
      elif today[i] == yesterday[i]:
          daily_chart.append([i, today[i], "same"])
      else:
          daily_chart.append([i, today[i], "down"])
    else:
      daily_chart.append([i, today[i], "new"])

dic_data = {
    'data': daily_chart
}
print(dic_data)

{'data': [['우크라', 35, 'up'], ['러시아', 28, 'up'], ['단일', 18, 'new'], ['결렬', 14, 'new'], ['윤석열', 14, 'up'], ['협상', 14, 'new'], ['키예프', 10, 'down'], ['금융', 10, 'new'], ['온화', 9, 'new'], ['전쟁', 9, 'new'], ['안철수', 9, 'new'], ['대선', 8, 'same'], ['일교차', 8, 'new'], ['제재', 8, 'down'], ['산불', 8, 'down'], ['확진', 8, 'down'], ['책임', 7, 'new'], ['강원', 7, 'new'], ['사태', 7, 'new'], ['유럽', 7, 'new'], ['국제', 7, 'new'], ['지원', 7, 'new'], ['휴일', 6, 'new'], ['반전', 6, 'new'], ['이재명', 6, 'new']]}


##weekly_chart

In [ ]:
cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= ADDDATE(curdate(), - WEEKDAY(curdate())) AND DATE(c_date) <= ADDDATE(curdate(), - WEEKDAY(curdate())+ 6) GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
keywords = cursor.fetchall()
weekly_chart = {}
if len(keywords) != 0:
  for i in range(5):
      cursor.execute(f"SELECT weight FROM keyword WHERE DATE(c_date) >= ADDDATE(curdate(), - WEEKDAY(curdate())) AND DATE(c_date) <= ADDDATE(curdate(), - WEEKDAY(curdate())+ 6) AND keyword = '{keywords[i][0]}' ORDER BY c_date;")
      daily_weight = list(cursor.fetchall())
      weight = [0]*7
      for n in range(len(daily_weight)):
        weight[n] = daily_weight[0][0]
      weekly_chart[keywords[i][0]] = weight
else:
  #이번주 데이터가 없을 시 지난주 데이터 추출
  cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= ADDDATE(date_add(curdate(), interval -1 day), - WEEKDAY(date_add(curdate(), interval -1 day))) AND DATE(c_date) <= ADDDATE(date_add(curdate(), interval -1 day), - WEEKDAY(date_add(curdate(), interval -1 day))+ 6) GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
  keywords = cursor.fetchall()
  for i in range(5):
      cursor.execute(f"SELECT weight FROM keyword WHERE DATE(c_date) >= ADDDATE(date_add(curdate(), interval -1 day), - WEEKDAY(date_add(curdate(), interval -1 day))) AND DATE(c_date) <= ADDDATE(date_add(curdate(), interval -1 day), - WEEKDAY(date_add(curdate(), interval -1 day))+ 6) AND keyword = '{keywords[i][0]}' ORDER BY c_date;")
      daily_weight = list(cursor.fetchall())
      weight = [0]*7
      for n in range(len(daily_weight)):
        weight[n] = daily_weight[0][0]
      weekly_chart[keywords[i][0]] = weight
dic_data = {
    'data': weekly_chart
}
print(dic_data)

{'data': {'우크라': [28, 28, 0, 0, 0, 0, 0], '러시아': [18, 18, 0, 0, 0, 0, 0], '키예프': [24, 24, 0, 0, 0, 0, 0], '윤석열': [9, 9, 0, 0, 0, 0, 0], '단일': [4, 4, 0, 0, 0, 0, 0]}}


##monthly_chart

In [ ]:
import calendar
from datetime import datetime
date = datetime.today().strftime("%Y%m%d")
last_day = calendar.monthrange(int(date[:4]), int(date[4:6]))[1]

cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= DATE_SUB(curdate(), INTERVAL (DAY(curdate())-1) DAY) AND DATE(c_date) <= LAST_DAY(NOW()) GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
keywords = cursor.fetchall()
monthly_chart = {}
if len(keywords) != 0:
  for i in range(5):
      cursor.execute(f"SELECT weight FROM keyword WHERE DATE(c_date) >= DATE_SUB(curdate(), INTERVAL (DAY(curdate())-1) DAY) AND DATE(c_date) <= LAST_DAY(NOW()) AND keyword = '{keywords[i][0]}' ORDER BY c_date;")
      daily_weight = list(cursor.fetchall())
      weight = [0] * last_day
      for n in range(len(daily_weight)):
        weight[n] = daily_weight[0][0]
        monthly_chart[keywords[i][0]] = weight
else:
  cursor.execute("SELECT keyword FROM keyword WHERE date_format(c_date, '%Y-%m') = date_format((curdate() - INTERVAL 1 MONTH), '%Y-%m') GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
  keywords = cursor.fetchall()
  for i in range(5):
    cursor.execute(f"SELECT weight FROM keyword WHERE date_format(c_date, '%Y-%m') = date_format((curdate() - INTERVAL 1 MONTH), '%Y-%m') AND keyword = '{keywords[i][0]}' ORDER BY c_date;")
    daily_weight = list(cursor.fetchall())
    weight = [0] * last_day
    for n in range(len(daily_weight)):
      weight[n] = daily_weight[0][0]
      monthly_chart[keywords[i][0]] = weight
dic_data = {
    'data': monthly_chart
}
print(dic_data)

{'data': {'우크라': [28, 28, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], '러시아': [18, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], '키예프': [24, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], '윤석열': [9, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], '단일': [4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}}


##daily_twitter

In [ ]:
cursor.execute("SELECT keyword FROM keyword WHERE c_date = date(now()) ORDER BY weight DESC LIMIT 5;")
keywords = list(cursor.fetchall())
if len(keywords) != 0:
  cursor.execute("SELECT keyword_id, CAST(SUM(retweet) AS SIGNED), CAST(SUM(like_count) AS SIGNED), group_concat(DISTINCT(nullif(tags, 'empty'))) FROM twitter WHERE SUBSTR(keyword_id, 1, 8) = date(now()) GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
  twitter = cursor.fetchall()
else:
  #오늘 정보 없을 시 어제 정보출력
  cursor.execute("SELECT keyword FROM keyword WHERE c_date = date_add(curdate(), interval -1 day) ORDER BY weight DESC LIMIT 5;")
  keywords = list(cursor.fetchall())
  cursor.execute("SELECT keyword_id, CAST(SUM(retweet) AS SIGNED), CAST(SUM(like_count) AS SIGNED), group_concat(DISTINCT(nullif(tags, 'empty'))) FROM twitter WHERE SUBSTR(keyword_id, 1, 8) = date_add(curdate(), interval -1 day) GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
  twitter = cursor.fetchall()

daily_twitter = {}
no = 1
while len(keywords) > 0:
    word = keywords.pop(0)
    for tweet in twitter:
        if word[0] in tweet[0]:
            daily_twitter[word[0]] = tweet[1:]
    no += 1
print(daily_twitter)

{'우크라': (43530, 43, 'SBS뉴스,우크라이나, 러시아, 정전협상, 평화협상, 회담'), '러시아': (257924, 155, 'PrayForUkraine,SBS뉴스'), '단일': (8105, 742, '안철수소신투표, 기호4번'), '윤석열': (59412, 4, '양아치,윤석열,팩트체크'), '결렬': (3473, 71, '대선, 단일화, 윤석열, TV토론, 안철수,안철수소신투표, 기호4번')}


##weekly_twitter

In [ ]:
cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= ADDDATE(curdate(), - WEEKDAY(curdate())) AND DATE(c_date) <= ADDDATE(curdate(), - WEEKDAY(curdate())+ 6) GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
keywords = list(cursor.fetchall())
if len(keywords) != 0:
  cursor.execute("SELECT keyword_id, CAST(SUM(retweet) AS SIGNED), CAST(SUM(like_count) AS SIGNED), group_concat(DISTINCT(nullif(tags, 'empty'))) FROM twitter WHERE SUBSTR(keyword_id, 1, 8) >= ADDDATE(curdate(), - WEEKDAY(curdate())) AND SUBSTR(keyword_id, 1, 8) <= ADDDATE(curdate(), - WEEKDAY(curdate())+ 6) GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
  twitter = cursor.fetchall()
else:
  cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= ADDDATE(date_add(curdate(), interval -1 day), - WEEKDAY(date_add(curdate(), interval -1 day))) AND DATE(c_date) <= ADDDATE(date_add(curdate(), interval -1 day), - WEEKDAY(date_add(curdate(), interval -1 day))+ 6) GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
  keywords = list(cursor.fetchall())
  cursor.execute("SELECT keyword_id, CAST(SUM(retweet) AS SIGNED), CAST(SUM(like_count) AS SIGNED), group_concat(DISTINCT(nullif(tags, 'empty'))) FROM twitter WHERE SUBSTR(keyword_id, 1, 8) >= ADDDATE(date_add(curdate(), interval -1 day), - WEEKDAY(date_add(curdate(), interval -1 day))) AND SUBSTR(keyword_id, 1, 8) <= ADDDATE(date_add(curdate(), interval -1 day), - WEEKDAY(date_add(curdate(), interval -1 day))+ 6) GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
  twitter = cursor.fetchall()

weekly_twitter = {}
for i in keywords:
  weekly_twitter[i[0]] = [0, 0, '']
no = 1
while len(keywords) > 0:
    word = keywords.pop(0)
    for tweet in twitter:
        if word[0] in tweet[0]:
            weekly_twitter[word[0]][0] += tweet[1]
            weekly_twitter[word[0]][1] += tweet[2]
            weekly_twitter[word[0]][2] += tweet[3]
    no += 1
print(weekly_twitter)

{'우크라': [86876, 86, 'SBS뉴스,우크라이나, 러시아, 정전협상, 평화협상, 회담SBS뉴스,우크라이나, 러시아, 정전협상, 평화협상, 회담'], '러시아': [491066, 305, 'PrayForUkraine,SBS뉴스PrayForUkraine,SBS뉴스'], '키예프': [294166, 11, 'SBS뉴스SBS뉴스'], '윤석열': [129205, 8, '양아치,윤석열,팩트체크양아치,윤석열,팩트체크'], '단일': [8105, 742, '안철수소신투표, 기호4번']}


##monthly_twitter

In [ ]:
cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= DATE_SUB(curdate(), INTERVAL (DAY(curdate())-1) DAY) AND DATE(c_date) <= LAST_DAY(NOW()) GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
keywords = list(cursor.fetchall())
if len(keywords) != 0:
  cursor.execute("SELECT keyword_id, CAST(SUM(retweet) AS SIGNED), CAST(SUM(like_count) AS SIGNED), group_concat(DISTINCT(nullif(tags, 'empty'))) FROM twitter WHERE SUBSTR(keyword_id, 1, 8) >= DATE_SUB(curdate(), INTERVAL (DAY(curdate())-1) DAY) AND SUBSTR(keyword_id, 1, 8) <= LAST_DAY(NOW()) GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
  twitter = cursor.fetchall()
else:
  #이번달 정보 없을 시 지난달 정보 출력
  cursor.execute("SELECT keyword FROM keyword WHERE date_format(c_date, '%Y-%m') = date_format((curdate() - INTERVAL 1 MONTH), '%Y-%m') GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
  keywords = list(cursor.fetchall())
  cursor.execute("SELECT keyword_id, CAST(SUM(retweet) AS SIGNED), CAST(SUM(like_count) AS SIGNED), group_concat(DISTINCT(nullif(tags, 'empty'))) FROM twitter WHERE SUBSTR(keyword_id, 1, 8) = date_format((curdate() - INTERVAL 1 MONTH), '%Y-%m') GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
  twitter = cursor.fetchall()

monthly_twitter = {}
for i in keywords:
  monthly_twitter[i[0]] = [0, 0, '']

no = 1
while len(keywords) > 0:
    word = keywords.pop(0)
    for tweet in twitter:
        if word[0] in tweet[0]:
            monthly_twitter[word[0]][0] += tweet[1]
            monthly_twitter[word[0]][1] += tweet[2]
            monthly_twitter[word[0]][2] += tweet[3]
    no += 1
print(monthly_twitter)

{'우크라': [86876, 86, 'SBS뉴스,우크라이나, 러시아, 정전협상, 평화협상, 회담SBS뉴스,우크라이나, 러시아, 정전협상, 평화협상, 회담'], '러시아': [491066, 305, 'PrayForUkraine,SBS뉴스PrayForUkraine,SBS뉴스'], '키예프': [294166, 11, 'SBS뉴스SBS뉴스'], '윤석열': [129205, 8, '양아치,윤석열,팩트체크양아치,윤석열,팩트체크'], '단일': [8105, 742, '안철수소신투표, 기호4번']}


##daily_youtube

In [ ]:
cursor.execute("SELECT keyword FROM keyword WHERE c_date = date(now()) ORDER BY weight DESC LIMIT 5;")
keywords = list(cursor.fetchall())
if len(keywords) != 0:
  cursor.execute("SELECT keyword_id, CAST(SUM(view_count) AS SIGNED), CAST(SUM(like_count) AS SIGNED), CAST(SUM(comment_count) AS SIGNED), group_concat(tags) FROM youtube WHERE SUBSTR(keyword_id, 1, 8) = curdate() GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
  youtube = cursor.fetchall()
else:
  #오늘 정보 없을 시 어제 정보 출력
  cursor.execute("SELECT keyword FROM keyword WHERE c_date = date_add(curdate(), interval -1 day) ORDER BY weight DESC LIMIT 5;")
  keywords = list(cursor.fetchall())
  cursor.execute("SELECT keyword_id, CAST(SUM(view_count) AS SIGNED), CAST(SUM(like_count) AS SIGNED), CAST(SUM(comment_count) AS SIGNED), group_concat(tags) FROM youtube WHERE SUBSTR(keyword_id, 1, 8) = date_add(curdate(), interval -1 day) GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
  youtube = cursor.fetchall()

daily_youtube = {}
no = 1
while len(keywords) > 0:
    word = keywords.pop(0)
    for data in youtube:
        if word[0] in data[0]:
            daily_youtube[word[0]] = data[1:]
    no += 1
print(daily_youtube)

{'우크라': (8249461, 72202, 32258, 'source:영상, type:방송, genre:사회, format:기타, person:박기완,source:영상, type:방송, genre:국제, format:인터뷰, person:호준석,source:영상, type:방송, genre:국제, format:기타,JTBC, JTBC NEWS, 뉴스, 뉴스룸, 손석희, newsroom, sohnsukhee, 제이티비씨, 제이티비씨 뉴스, ㅓ슟, 제이티비시, 제이티비시 뉴스, 안나경, 한민용, 오대영,YTN실시간, YTN, 우크라이나, 미국, 러시아, 영국, 사이버공격, 비상사태,SBSBiz, 경제, 뉴스,source:영상, type:방송, genre:국제, format:기타, person:구수본,MBC, MBC뉴스, 뉴스데스크, newsdesk, 뉴스투데이, newstoday, 8시뉴스, 아침뉴스, 뉴스, 정오뉴스, news, 러시아, 우크라이나, 키예프, 전쟁, 블라디미르 푸틴, 볼로디미르 젤렌스키,MBC, MBC뉴스, 뉴스데스크, newsdesk, 뉴스투데이, newstoday, 8시뉴스, 아침뉴스, 뉴스, 정오뉴스, news, 볼로디미르 젤렌스키, 우크라이나, 러시아, 블라디미르 푸틴,sourc'), '러시아': (28240321, 152542, 30699, '금융, 제재, 핵폭탄,MBC, MBC뉴스, 뉴스데스크, newsdesk, 뉴스투데이, newstoday, 8시뉴스, 아침뉴스, 뉴스, 정오뉴스, news, 우크라이나, 키예프, 전쟁, 러시아, Kyiv, Ukraine, Livecam, Russia, Putin,슈카, 슈카월드, 경제, 시사, 이슈, 뉴스, 상식, 면접, 논술, 자소서, 취업, 주식, 증시, 주가, 시장, 자산, 투자, 금리,source:영상, type:방송, genre:국제, format:기타, person:김원배,source:영상, type:방송, genre:국제, format:리포트, person:기정훈,KOREA NEWS, 

##weekly_youtube

In [ ]:
cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= ADDDATE(curdate(), - WEEKDAY(curdate())) AND DATE(c_date) <= ADDDATE(curdate(), - WEEKDAY(curdate())+ 6) ORDER BY weight DESC LIMIT 5;")
keywords = list(cursor.fetchall())
if len(keywords) != 0:
  cursor.execute("SELECT keyword_id, CAST(SUM(view_count) AS SIGNED), CAST(SUM(like_count) AS SIGNED), CAST(SUM(comment_count) AS SIGNED), group_concat(tags) FROM youtube WHERE SUBSTR(keyword_id, 1, 8) >= ADDDATE(curdate(), - WEEKDAY(curdate())) AND SUBSTR(keyword_id, 1, 8) <= ADDDATE(curdate(), - WEEKDAY(curdate())+ 6) GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
  youtube = cursor.fetchall()
else:
  #이번주 정보 없을 시 지난주 정보 출력
  cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= ADDDATE(date_add(curdate(), interval -1 day), - WEEKDAY(date_add(curdate(), interval -1 day))) AND DATE(c_date) <= ADDDATE(date_add(curdate(), interval -1 day), - WEEKDAY(date_add(curdate(), interval -1 day))+ 6) ORDER BY weight DESC LIMIT 5;")
  keywords = list(cursor.fetchall())  
  cursor.execute("SELECT keyword_id, CAST(SUM(view_count) AS SIGNED), CAST(SUM(like_count) AS SIGNED), CAST(SUM(comment_count) AS SIGNED), group_concat(tags) FROM youtube WHERE SUBSTR(keyword_id, 1, 8) >= ADDDATE(date_add(curdate(), interval -1 day), - WEEKDAY(date_add(curdate(), interval -1 day))) AND SUBSTR(keyword_id, 1, 8) <= ADDDATE(date_add(curdate(), interval -1 day), - WEEKDAY(date_add(curdate(), interval -1 day))+ 6) GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
  youtube = cursor.fetchall()

weekly_youtube = {}
for i in keywords:
  weekly_youtube[i[0]] = [0, 0, 0, '']
no = 1
while len(keywords) > 0:
    word = keywords.pop(0)
    for data in youtube:
        if word[0] in data[0]:
            weekly_youtube[word[0]][0] += data[1]
            weekly_youtube[word[0]][1] += data[2]
            weekly_youtube[word[0]][2] += data[3]
            weekly_youtube[word[0]][3] += data[4]
    no += 1
print(weekly_youtube)

{'우크라': [24472300, 211900, 102902, 'source:영상, type:방송, genre:사회, format:기타, person:박기완,source:영상, type:방송, genre:국제, format:인터뷰, person:호준석,source:영상, type:방송, genre:국제, format:기타,JTBC, JTBC NEWS, 뉴스, 뉴스룸, 손석희, newsroom, sohnsukhee, 제이티비씨, 제이티비씨 뉴스, ㅓ슟, 제이티비시, 제이티비시 뉴스, 안나경, 한민용, 오대영,YTN실시간, YTN, 우크라이나, 미국, 러시아, 영국, 사이버공격, 비상사태,SBSBiz, 경제, 뉴스,source:영상, type:방송, genre:국제, format:기타, person:구수본,MBC, MBC뉴스, 뉴스데스크, newsdesk, 뉴스투데이, newstoday, 8시뉴스, 아침뉴스, 뉴스, 정오뉴스, news, 러시아, 우크라이나, 키예프, 전쟁, 블라디미르 푸틴, 볼로디미르 젤렌스키,MBC, MBC뉴스, 뉴스데스크, newsdesk, 뉴스투데이, newstoday, 8시뉴스, 아침뉴스, 뉴스, 정오뉴스, news, 볼로디미르 젤렌스키, 우크라이나, 러시아, 블라디미르 푸틴,sourcJTBC, JTBC NEWS, 뉴스, 뉴스룸, 손석희, newsroom, sohnsukhee, 제이티비씨, 제이티비씨 뉴스, ㅓ슟, 제이티비시, 제이티비시 뉴스, 안나경, 한민용, 오대영, 러시아, 러시아군, 키예프, 우크라이나 침공, 러시아 침공, 푸틴, 바이든, 젤렌스키, 우크라이나 상황, No War Please, 우크라이나 필사 저항, 경비대, 도시 폭격, 러시아 군함 꺼져라,source:영상, type:방송, genre:국제, format:기타, person:김종균,JTBC, JTBC NEWS, 뉴스, 뉴스룸, 손석희, newsroom, sohnsukhee, 제이티비씨, 제이티비씨 뉴스, ㅓ슟, 제이티비시, 제이티비시 뉴스, 안나경, 한민용, 오대영

##monthly_youtube

In [ ]:
cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= DATE_SUB(curdate(), INTERVAL (DAY(curdate())-1) DAY) AND DATE(c_date) <= LAST_DAY(NOW()) ORDER BY weight DESC LIMIT 5;")
keywords = list(cursor.fetchall())
if len(keywords) != 0:
  cursor.execute("SELECT keyword_id, CAST(SUM(view_count) AS SIGNED), CAST(SUM(like_count) AS SIGNED), CAST(SUM(comment_count) AS SIGNED), group_concat(tags) FROM youtube WHERE published >= DATE_SUB(curdate(), INTERVAL (DAY(curdate())-1) DAY) AND published <= LAST_DAY(NOW()) GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
  twitter = cursor.fetchall()
else:
  cursor.execute("SELECT keyword FROM keyword WHERE date_format(c_date, '%Y-%m') = date_format((curdate() - INTERVAL 1 MONTH), '%Y-%m') ORDER BY weight DESC LIMIT 5;")
  keywords = list(cursor.fetchall()) 
  cursor.execute("SELECT keyword_id, CAST(SUM(view_count) AS SIGNED), CAST(SUM(like_count) AS SIGNED), CAST(SUM(comment_count) AS SIGNED), group_concat(tags) FROM youtube  WHERE SUBSTR(keyword_id, 1, 8) = date_format((curdate() - INTERVAL 1 MONTH), '%Y-%m') GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
  twitter = cursor.fetchall() 

monthly_youtube = {}
for i in keywords:
  monthly_youtube[i[0]] = [0, 0, 0, '']
no = 1
while len(keywords) > 0:
    word = keywords.pop(0)
    for data in youtube:
        if word[0] in data[0]:
            monthly_youtube[word[0]][0] += data[1]
            monthly_youtube[word[0]][1] += data[2]
            monthly_youtube[word[0]][2] += data[3]
            monthly_youtube[word[0]][3] += data[4]
    no += 1
print(monthly_youtube)

{'우크라': [24472300, 211900, 102902, 'source:영상, type:방송, genre:사회, format:기타, person:박기완,source:영상, type:방송, genre:국제, format:인터뷰, person:호준석,source:영상, type:방송, genre:국제, format:기타,JTBC, JTBC NEWS, 뉴스, 뉴스룸, 손석희, newsroom, sohnsukhee, 제이티비씨, 제이티비씨 뉴스, ㅓ슟, 제이티비시, 제이티비시 뉴스, 안나경, 한민용, 오대영,YTN실시간, YTN, 우크라이나, 미국, 러시아, 영국, 사이버공격, 비상사태,SBSBiz, 경제, 뉴스,source:영상, type:방송, genre:국제, format:기타, person:구수본,MBC, MBC뉴스, 뉴스데스크, newsdesk, 뉴스투데이, newstoday, 8시뉴스, 아침뉴스, 뉴스, 정오뉴스, news, 러시아, 우크라이나, 키예프, 전쟁, 블라디미르 푸틴, 볼로디미르 젤렌스키,MBC, MBC뉴스, 뉴스데스크, newsdesk, 뉴스투데이, newstoday, 8시뉴스, 아침뉴스, 뉴스, 정오뉴스, news, 볼로디미르 젤렌스키, 우크라이나, 러시아, 블라디미르 푸틴,sourcJTBC, JTBC NEWS, 뉴스, 뉴스룸, 손석희, newsroom, sohnsukhee, 제이티비씨, 제이티비씨 뉴스, ㅓ슟, 제이티비시, 제이티비시 뉴스, 안나경, 한민용, 오대영, 러시아, 러시아군, 키예프, 우크라이나 침공, 러시아 침공, 푸틴, 바이든, 젤렌스키, 우크라이나 상황, No War Please, 우크라이나 필사 저항, 경비대, 도시 폭격, 러시아 군함 꺼져라,source:영상, type:방송, genre:국제, format:기타, person:김종균,JTBC, JTBC NEWS, 뉴스, 뉴스룸, 손석희, newsroom, sohnsukhee, 제이티비씨, 제이티비씨 뉴스, ㅓ슟, 제이티비시, 제이티비시 뉴스, 안나경, 한민용, 오대영